In [1]:
import os
import glob
from dataclasses import dataclass
import logging
logging.basicConfig(level=logging.INFO, 
    format='%(asctime)s %(levelname)s - %(message)s',
    datefmt="%H:%M:%S"
)

import numpy as np
from shutil import rmtree

def clean():
    for name in ("my-experiment", "ioh_data"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)

def ls(p="./"):
    for obj in os.listdir(p):
        print(obj)

def cat(f):
    with open(f) as h:
        print(h.read())

clean()

In [2]:

import ioh
%pip show ioh

Name: ioh
Version: 0.3.3
Summary: The experimenter for Iterative Optimization Heuristics
Home-page: None
Author: Jacob de Nobel, Furong Ye, Diederick Vermetten, Hao Wang, Carola Doerr and Thomas Bäck
Author-email: iohprofiler@liacs.leidenuniv.nl
License: None
Location: /home/jacob/code/IOHexperimenter
Requires: numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Problem
In ioh, everything revolves around the `problem` class, which exists for both `Real` and `Integer` types for continious and discrete problems resp. These classes are wrappers around an objective function, and can be used to interact with the various other parts of iohexperimenter. 

We have a number of objective functions already implemented for convenience, which includes the objective functions from the BBOB single objective benchmark by the COCO platform for the continous case and the PBO benchmark functions for the discrete case.


In [3]:
# A list of problems can be accessed via the base classes
ioh.problem.Real.problems

{1: 'Sphere',
 2: 'Ellipsoid',
 3: 'Rastrigin',
 4: 'BuecheRastrigin',
 5: 'LinearSlope',
 6: 'AttractiveSector',
 7: 'StepEllipsoid',
 8: 'Rosenbrock',
 9: 'RosenbrockRotated',
 10: 'EllipsoidRotated',
 11: 'Discus',
 12: 'BentCigar',
 13: 'SharpRidge',
 14: 'DifferentPowers',
 15: 'RastriginRotated',
 16: 'Weierstrass',
 17: 'Schaffers10',
 18: 'Schaffers1000',
 19: 'GriewankRosenBrock',
 20: 'Schwefel',
 21: 'Gallagher101',
 22: 'Gallagher21',
 23: 'Katsuura',
 24: 'LunacekBiRastrigin'}

In [4]:
# In order to instantiate a problem instance, we can do the following:
problem = ioh.get_problem(
    "Sphere", 
    instance=1,
    dimension=10,
    problem_type="Real"
)
problem

<RealProblem 1. Sphere (iid=1 dim=10)>

In [5]:
# The problem class includes information about the problem, which can be retrieved via the meta_data accessor
problem.meta_data.name

'Sphere'

In [6]:
# The current state of the problem, e.g. the number of evaluations, best seen points etc. are stored in the problems state.
problem.state

<State evaluations: 0 optimum_found: false current_best: <Solution x: {nan, nan, nan, nan, nan, nan, nan, nan, nan, nan} y: nan>>

In [7]:
# Every problem as has a simple box-bounds associcated
problem.bounds

<BoxConstraint lb: [{-5, -5, -5, -5, -5, -5, -5, -5, -5, -5}] ub: [{5, 5, 5, 5, 5, 5, 5, 5, 5, 5}]>

In [8]:
# We can access the contraint information of the problem
x0 = np.random.uniform(problem.bounds.lb, problem.bounds.ub)

# Evaluation happens like a 'normal' objective function would
problem(x0)

# Whenever the problem is evaluated, the state changes
problem.state

<State evaluations: 1 optimum_found: false current_best: <Solution x: {2.9096648038060895, -4.274905893515328, -2.8901093183139013, 2.183573703955002, 2.921525281291302, 2.7140942864471738, -2.840045244582139, -0.9797296643737852, 2.4776647455303547, -1.4118865941578695} y: 165.82909777889816>>

In [9]:
# If we want to perform multiple runs with the same objective function, after every run, the problem has to be reset, 
# such that the internal state reflects the current run.
def run_experiment(problem, algorithm, n_runs=5):
    for run in range(n_runs):
        
        # Run the algorithm on the problem
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found:{problem.state.current_best.y: .3f}")

        # Reset the problem
        problem.reset()

In [10]:
@dataclass
class RandomSearch:
    'Simple random search algorithm'
    
    n: int
    length: float = 0.0
        
    def __call__(self, problem: ioh.problem.Real) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems bounds accessor to get information about the problem bounds
            x = np.random.uniform(problem.bounds.lb, problem.bounds.ub)
            self.length = np.linalg.norm(x)
            
            problem(x)  
            

In [11]:
# using the random search algorithm, we can then run a simple experiment
run_experiment(problem, RandomSearch(10))

run: 1 - best found: 142.534
run: 2 - best found: 131.735
run: 3 - best found: 165.210
run: 4 - best found: 139.593
run: 5 - best found: 114.560


# Adding custom problems
As the list of already implemented problems might not contains the objective function you would like to analyze and benchmark, we include an easy to use interface to wrap your benchmark function into an iohexperimenter `problem` object. 

Currently, only single objective functions are supported, so the only requirement on the function is its signature, which should take a single array parameter, and return a single floating point number $f(\mathbf{x}) \mapsto \mathbb{R}$.

In this example we will add the function (Styblinski–Tang):

$f(\mathbf{x}) = \frac{\sum_{i=1}^n x_i^4 - 16x_i^2 + 5x_i}{2}$

defined on $[-5, 5]$

In [12]:
def styblinski_tang(x: np.ndarray) -> float:
    return np.sum(np.power(x, 4) - (16 * np.power(x, 2)) + (5 * x)) / 2


styblinski_tang(np.array([-2.903534]*10)) # global minima

-391.661657037714

In [13]:
# we can wrap this function in ioh, as this is is a continous function, we use wrap_real_problem:
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     # Handle to the function
    name="StyblinskiTang",                               # Name to be used when instantiating
    optimization_type=ioh.OptimizationType.MIN, # Specify that we want to minimize
    lb=-5,                                               # The lower bound
    ub=5,                                                # The upper bound
)

In [14]:
# We can create an instance of this problem, wrapped in ioh
problem = ioh.get_problem("StyblinskiTang", dimension=10)
problem

<RealProblem 25. StyblinskiTang (iid=1 dim=10)>

In [15]:
problem(np.array([-2.903534]*10))

-391.661657037714

## Specifying instances
When doing bechmarking of custom problems, it can often be usefull to look at different instances of the same problem, but that perform some transformation the parameter or objective space, to test if an algorithm is invariant to such transformations. 

Here we add the following transformations to the aforementioned objective function:

$T_x(\mathbf{x}, c) = \mathbf{x} + c$

$T_y(y, c) = y * c$

We can do this by providing transformation functions to the `wrap_problem` interface. Note that these take two parameters:
 1. The operand they function on, which are the variables for the variables transformation and objective function value for the objective transformation. 
 2. An integer identifier of the instance id which is currently used. This allows you to specify alternate behavior for different instances.  

In [16]:
# Variables transformation R^d -> R^d
def transform_variables(x: np.ndarray, instance_id:int) -> np.ndarray:
    c = (instance_id - 1) * .5
    return x + c

# Objective transformation R -> R
def transform_objectives(y: float, instance_id:int) -> float:
    c = instance_id
    return y * c

# Note that we can overwrite a previously defined problem by calling wrap_real_problem again with the same name
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     
    name="StyblinskiTang",                               
    optimization_type=ioh.OptimizationType.MIN, 
    lb=-5,                                               
    ub=5,      
    
    # Adding the transformation functions
    transform_variables=transform_variables,     
    transform_objectives=transform_objectives
)

In [17]:
# We can now create different instances of the same problem
instance1 = ioh.get_problem("StyblinskiTang", instance=1, dimension=10)
instance2 = ioh.get_problem("StyblinskiTang", instance=2, dimension=10)
instance1, instance2

(<RealProblem 25. StyblinskiTang (iid=1 dim=10)>,
 <RealProblem 25. StyblinskiTang (iid=2 dim=10)>)

In [18]:
# Note that when evaluating with the same point, each instance gives a different (transformed value)
instance1(x0), instance2(x0)

(-214.89621584535837, -414.8219175071203)

# Logging data
The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer. 

In [19]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

<Analyzer /home/jacob/code/IOHexperimenter/example/my-experiment>

In [20]:
ls()

add_new_problem.cpp
README.md
CMakeLists.txt
gsemo.cpp
eafh.cpp
run_submodular_instances.cpp
main.cpp
tutorial.ipynb
suite_example.h
experiment_example.h
problem_example.h
logger_example.h
my-experiment
CMakeFiles


In [21]:
# In order to log data for a problem, we only have to attach it to a logger
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)

# We can then run the random search as before, only now all data will be logged to a file
run_experiment(problem, RandomSearch(10), n_runs=1)

# Force logger to flush and write files. (> v0.3.3)
# This is only required when using the logger in an interactive context, such as this jupyter notebook.
# Otherwise, the destructor, which is called on exit of the python interpreter, forces such a write automatically.
logger.close()

run: 1 - best found:-50.778


In [22]:
cat("./my-experiment/IOHprofiler_f25_StyblinskiTang.json")

{
	"version": "0.3.3", 
	"suite": "unknown_suite", 
	"function_id": 25, 
	"function_name": "StyblinskiTang", 
	"maximization": false, 
	"algorithm": {"name": "random-search", "info": "algorithm_info"},
	"attributes": ["evaluations", "raw_y"],
	"scenarios": [
		{"dimension": 2,
		"path": "data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat",
		"runs": [
			{"instance": 1, "evals": 10, "best": {"evals": 1, "y": -50.77844171427786, "x": [-3.6361032429010844, 3.0691931784415516]}}
		]}
	]
}



In [23]:
cat("my-experiment/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat")

evaluations raw_y x0 x1
1 -50.7784417143 -3.636103 3.069193
10 79.9690893779 3.286221 4.845003



## Triggers 
The default behavior of the `Analyzer` logger is to log data only when there is an improvement of the objective value. We can change this behaviour, by specifying one or more triggers, which are logical operators, which when one of them evaluates to True, will cause data to be logged.

We provide a number of trigger variants which can be use to customize the logging. In the following example, a trigger is defined which evaluates to True, every 3 function evaluations. It is combined with a trigger for improvement, so data will be logged on every 3rd function evaluation, or when there is an observed improvement of the objective value

In [24]:
triggers = [
    ioh.logger.trigger.Each(3),
    ioh.logger.trigger.OnImprovement()
]

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True,
    
    # Add the triggers to the logger
    triggers = triggers
)

logger

<Analyzer /home/jacob/code/IOHexperimenter/example/my-experiment-1>

In [25]:
# Rerun the same experiment as before
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)
run_experiment(problem, RandomSearch(10), n_runs=1)
logger.close()

run: 1 - best found:-50.107


In [26]:
# We can now see that data is logged either if there is improvement, or on every 3rd evaluation
cat("my-experiment-1/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat")

evaluations raw_y x0 x1
1 -50.1070679124 -1.264886 -3.337073
3 -47.8029600624 2.816379 -3.753524
6 -44.7448409335 -2.543703 1.205442
9 55.4169524805 3.071565 -4.865833
10 -32.2965616007 -2.169564 -0.091210



## Properties
If we want to keep track of any dynamic parameters a given algorithm might have, we can use properties to log them to the output files. 

In the following example, we will track the length parameters for the RandomSearch algorithm, which is added for illustrative purpoposes, and changes for every function evaluation

In [27]:
# RandomSearch has a length parameter, which is dynamic
algorithm = RandomSearch(10)

# Creating a new logger
logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True
)

# Before we attach a problem, we tell the logger to keep track of the length parameter on algorithm
logger.watch(algorithm, "length")

# We can now again run the same experiment 
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)

problem.attach_logger(logger)
run_experiment(problem, algorithm, n_runs=1)
logger.close()

run: 1 - best found:-56.134


In [28]:
# Note the additional length parameter being logged 
cat("my-experiment-2/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat")

evaluations raw_y length x0 x1
1 -34.6243233839 3.2428298000 1.390234 2.929709
4 -42.3394191997 4.5655699591 -2.197340 -4.002015
7 -49.2610922803 2.9092639431 -1.971292 2.139585
8 -56.1338583576 4.6540636306 3.228611 -3.352071
10 -18.8493130227 2.0114686140 0.368748 1.977380



# Alternate logging behaviour
We provide a number of different loggers in addition to the `Analyzer` logger, which include:
 - `FlatFile` which logs data to a simple csv file
 - `Store` which keeps all of the stored data in memory
 - `EAF/EAH` which compute Empirical Attainment Function/Histogram statistics on the fly

You can define your own custom logging behavoir by inheriting from the `AbstractLogger` class. The only required part of the interface is that you override the `__call__` operator, which takes a single `ioh.LogInfo` parameter. In this method you should define your desired behavior. 

In [29]:
# Simple logger that logs data using the python logging module whenever it is triggeredd
class MyLogger(ioh.logger.AbstractLogger):
    def __call__(self, log_info: ioh.LogInfo):
        logging.info(msg=f"triggered! y: {log_info.y}")


# The abstract logger takes two parameters, triggers and properties
mylogger = MyLogger(triggers=[ioh.logger.trigger.ALWAYS])

problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(mylogger)

run_experiment(problem, RandomSearch(10), 1)

04:59:04 INFO - triggered! y: -36.64162472946894
04:59:04 INFO - triggered! y: -27.89978901407425
04:59:04 INFO - triggered! y: -43.739066310909536
04:59:04 INFO - triggered! y: -10.160644903707155
04:59:04 INFO - triggered! y: -26.487843911262406
04:59:04 INFO - triggered! y: 65.8107866193718
04:59:04 INFO - triggered! y: -38.08550743236062
04:59:04 INFO - triggered! y: -28.838895692089366
04:59:04 INFO - triggered! y: -30.67165198237271
04:59:04 INFO - triggered! y: -43.46970117771531


run: 1 - best found:-43.739


# Standardized Experimental Setup (Python only)
In Python, we provide the `Experiment` class which can be used to easily run a given algorithm over a larger number of problems. 

In [30]:
experiment = ioh.Experiment(
    algorithm = RandomSearch(10), # An algorithm instance
    fids = [1, 25],               # the id's of the problems we want to test
    iids = [1, 10],               # the instances 
    dims = [2, 10],               # the dimensions
    reps = 3,                     # the number of runs,
    zip_output = True       
)

In [31]:
experiment.run()

In [32]:
ls("./ioh_data")

IOHprofiler_f25_StyblinskiTang.info
data_f25_StyblinskiTang
data_f1_Sphere
IOHprofiler_f1_Sphere.info


In [33]:
cat("ioh_data/IOHprofiler_f25_StyblinskiTang.info")

suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 10, maximization = "F", algId = "RandomSearch(n=10, length=0.0)", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM10.dat, 10:9|945.132, 10:9|2509.12, 10:7|1465.29
suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 10, maximization = "F", algId = "RandomSearch(n=10, length=0.0)", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM10.dat, 1:10|-219.877, 1:10|-221.724, 1:8|-160.455
suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 2, maximization = "F", algId = "RandomSearch(n=10, length=0.0)", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat, 1:6|-47.4678, 1:1|-49.6946, 1:6|-60.802
suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 2, maximization = "F", algId = "RandomSearch(n=10, length=0.0)", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat, 10:1

In [34]:
clean()

In [35]:
# cleanup
rmtree("my-experiment", ignore_errors=True)
rmtree("ioh_data", ignore_errors=True)

# Working with constraints (> v0.3.3)
Every problem has a set of constraints associated with it. For most of the currently implemented problems, this is an empty set by default. However, we can modify this set, and add arbitrary functions as constraints. 

In [36]:
# We take the Shere function as example
p = ioh.get_problem("Sphere", 1, 2)

# The set of constraints is empty by default
p.constraints


<ConstraintSet: >

In [37]:
# We can turn the bounds of the problem into a contraint
print(p.bounds)

# By default, bounds are not enforced, so violating them changes nothing to the returned objective function value
p([10, 10])


<BoxConstraint lb: [{-5, -5}] ub: [{5, 5}]>


298.96209408

In [38]:
# For bound constraints, violation is computed as the sum of the squared violation per component
p.enforce_bounds(
    how=ioh.ConstraintEnforcement.SOFT, # The enforcement strategy: SOFT ensure penalization on violation by a penalty (y + p)
    weight=1.0,                         # Penaly p is computed as: weight * violation ^ exponent                       
    exponent=1.0,                        
)
# Now the bounds are in the constraint set
p.constraints

<ConstraintSet: <BoxConstraint lb: [{-5, -5}] ub: [{5, 5}]>>

In [39]:
# We can now observe the added penalty term
p([10, 10]) 

348.96209408

In [40]:
# There a several strategies that modify a constraint's behavoir:
types = (
    ioh.ConstraintEnforcement.NOT,       # Don't calulate the constraint function
    ioh.ConstraintEnforcement.HIDDEN,    # Calulate the constraint, but don't penalize
    ioh.ConstraintEnforcement.SOFT,      # Calculate both constraint and objective function value y, and penalize as y + p
    ioh.ConstraintEnforcement.HARD,      # Calulate the constraint, if there is violation, don't calculate y, and only return p
)
for strategy in types:
    p.enforce_bounds(how=strategy, weight=1.0, exponent=1.0)
    print(strategy, p([10, 10]), p.constraints.violation())


ConstraintEnforcement.NOT 298.96209408 0.0
ConstraintEnforcement.HIDDEN 298.96209408 50.0
ConstraintEnforcement.SOFT 348.96209408 50.0
ConstraintEnforcement.HARD 50.0 50.0


In [41]:
# Note that when we use a HARD method, the objective value is actually lower (50.0) than the actual objective value (298.96)
# This is because we didn't change any of the other parameters, that compute the penalty. Normally, with HARD constraints, we 
# want to also make sure that we return a y value that is infeasible. We can do this by changing the weight and/or exponent parameters.
p.enforce_bounds(
    how=ioh.ConstraintEnforcement.HARD, 
    weight=float("inf"),                
    exponent=1.0                        
)

# Any infeasible solution gets an infinity value
p([10, 10])


inf

In [42]:
# Note that the actual objective function is not called for infeasible points with HARD constraints:
p.state.y_unconstrained

inf

In [43]:
# If we pass a feasible point it is evaluated
p([1.,-1.]), p.state.y_unconstrained

(80.06289408, 80.06289408)

### Custom constraint functions

In [44]:
# We can create custom constraint functions, which can be 'just' functions that check something on x.
# Such functions should return a nonzero floating point value when there is violation, and zero otherwise. 

def is_x_ordered(x: np.ndarray) -> float:
    '''Checks that xi < xi+1'''

    return 1. - (np.diff(x) > 0).all().astype(float)

# x is not ordered, so there is a violation (1.)
print(is_x_ordered([10, 1, 3]))

# x is ordered, so there is no violation (0.)
print(is_x_ordered([1, 3, 10]))

1.0
0.0


In [45]:
# We can add such a function to an arbitary problem as constraint like so:
constraint = ioh.RealConstraint(is_x_ordered, name="is_x_ordered", weight=100_000)

p = ioh.get_problem("Sphere", 1, 3)
p.add_constraint(constraint)

# Note that is gets added to the list of constraints. 
p.constraints 

<ConstraintSet: <FunctionalConstraint is_x_ordered>>

In [46]:
print(p([4, 1, 3]))  # The penalty gets added 
print(p([1, 3, 4]))  # The penalty doesn't get added

100112.04147008
119.63347008000001


In [47]:
# We can remove the contraint again
p.remove_constraint(constraint)
p.constraints

<ConstraintSet: >

### Wrapping constraints

In [48]:
# We can also add constraints to the problem definition to wrap problem
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     
    name="StyblinskiTang",                               
    optimization_type=ioh.OptimizationType.MIN, 
    lb=-5,                                              
    ub=5,   
    # Adding the constraints                                            
    constraints=[constraint]
)

In [49]:
# Now if we instantiate an instance of the Styblinski-Tang function, is has the x_ordering constraint added automatically
p = ioh.get_problem("StyblinskiTang", 1, 5)
p.constraints

<ConstraintSet: <FunctionalConstraint is_x_ordered>>

### Logging constraints

In [50]:
# Creating a new logger
logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True,
    
    # We add the constraint properties to the list of logged items
    # Note that these properties are aggregated values for ALL applied constrains, 
    # So when you have more than one constraint added to a problem, these values will be the sums
    # of all the applied constraints. 
    additional_properties=[
        ioh.logger.property.CURRENTY,   # The constrained y-value, by default only the untransformed & unconstraint y
                                        # value is logged. 
        ioh.logger.property.VIOLATION,  # The raw violation value
        ioh.logger.property.PENALTY     # The applied penalty
    ]
)

# We can now again run the same experiment 
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)

problem.attach_logger(logger)
run_experiment(problem, RandomSearch(10), n_runs=1)
logger.close()

run: 1 - best found:-46.698


In [51]:
cat("my-experiment/IOHprofiler_f25_StyblinskiTang.json")
cat("my-experiment/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat")

{
	"version": "0.3.3", 
	"suite": "unknown_suite", 
	"function_id": 25, 
	"function_name": "StyblinskiTang", 
	"maximization": false, 
	"algorithm": {"name": "random-search", "info": "algorithm_info"},
	"attributes": ["evaluations", "raw_y", "current_y", "violation", "penalty"],
	"scenarios": [
		{"dimension": 2,
		"path": "data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat",
		"runs": [
			{"instance": 1, "evals": 10, "best": {"evals": 10, "y": -69.67715120733354, "x": [-2.468299100152127, -3.436892564131088]}}
		]}
	]
}

evaluations raw_y current_y violation penalty x0 x1
1 -33.4929461882 99966.5070538118 1.0000000000 100000.0000000000 -2.830703 -4.220391
2 -46.6981091541 -46.6981091541 0.0000000000 0.0000000000 2.312842 3.000766
7 -60.2135850162 99939.7864149838 1.0000000000 100000.0000000000 -3.484146 -3.618621
10 -69.6771512073 99930.3228487927 1.0000000000 100000.0000000000 -2.468299 -3.436893



In [52]:
clean()
